In [0]:
from pyspark.sql.functions import sum, count, avg, ntile, col
from pyspark.sql.window import Window

sales = spark.table("project_4.silver.sales")

# LTV calculation
customer_ltv = (
    sales.groupBy("customer_unique_id")
         .agg(
             count("order_id").alias("total_orders"),
             sum("price").alias("total_spent"),
             avg("price").alias("avg_order_value")
         )
)

# RFM-style spend ranking
window = Window.orderBy(col("total_spent").desc())
customer_ltv = customer_ltv.withColumn("spend_rank", ntile(5).over(window))

customer_ltv.write.mode("overwrite").saveAsTable("project_4.gold.customer_lifetime_value")

print("✅ gold.customer_lifetime_value created successfully")
